In [50]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import cv2, 
import random
import json
import csv
import matplotlib.pyplot as plt

%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras import optimizers

import os


In [51]:
np.random.seed(1337)


In [62]:
TRAIN_DIR = './cats_and_dogs_filtered/train/'
TEST_DIR = './cats_and_dogs_filtered/validation/'

ROWS = 150
COLS = 150
CHANNELS = 3

BATCH_SIZE=128

In [101]:
# Criando uma lista das imagens de treino totais e listas separadas para cada animal
original_train_images=[]
for i in os.listdir(TRAIN_DIR):
    original_train_images.append(os.listdir(TRAIN_DIR+i))
    if i == 'dogs':
        train_dogs = os.listdir(TRAIN_DIR+i)
    if i == 'cats':
        train_cats = os.listdir(TRAIN_DIR+i)
original_train_images = original_train_images[0] + original_train_images[1]
random.shuffle(original_train_images)

In [102]:
# Concatenando o diretorio das imagens para preservar o diretorio
for i in range(len(original_train_images)):
    if 'dog' in original_train_images[i]:
        original_train_images[i] = TRAIN_DIR + 'dogs/'+original_train_images[i]
    else:
        original_train_images[i] = TRAIN_DIR + 'cats/'+original_train_images[i]

In [103]:
original_train_images

['./cats_and_dogs_filtered/train/cats/cat.700.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.772.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.329.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.475.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.855.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.993.jpg',
 './cats_and_dogs_filtered/train/cats/cat.974.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.179.jpg',
 './cats_and_dogs_filtered/train/cats/cat.311.jpg',
 './cats_and_dogs_filtered/train/cats/cat.388.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.214.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.398.jpg',
 './cats_and_dogs_filtered/train/cats/cat.428.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.261.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.764.jpg',
 './cats_and_dogs_filtered/train/cats/cat.491.jpg',
 './cats_and_dogs_filtered/train/cats/cat.733.jpg',
 './cats_and_dogs_filtered/train/dogs/dog.522.jpg',
 './cats_and_dogs_filtered/train/cats/cat.882.jpg',
 './cats_and

In [85]:
test_images=[]
for i in os.listdir(TEST_DIR):
    test_images.append(os.listdir(TEST_DIR+i))
test_images = test_images[0] + test_images[1]

In [86]:
def plot_arr(arr):
    plt.figure()
    plt.imshow(image.array_to_img(arr))
    plt.show()

def plot(img):
    plt.figure()
    plt.imshow(img)
    plt.show()

In [118]:
def prep_data(images):
    count = len(images)
    X = np.ndarray((count, ROWS, COLS, CHANNELS), dtype=np.float32)
    y = np.zeros((count,), dtype=np.float32)
    print("Starting")
    for i, image_file in enumerate(images):
        img = image.load_img(image_file, target_size=(ROWS, COLS))
        X[i] = image.img_to_array(img)
        if 'dog.' in image_file:
            y[i] = 1.
        i+=1
        if i%1000 == 0: print('Processed {} of {}'.format(i, count))
    return X, y


In [116]:
X_train, y_train = prep_data(original_train_images)

Starting
Processed 1000 of 2000
Processed 2000 of 2000


In [99]:
train_datagen = image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

In [119]:
train_generator = train_datagen.flow(
    X_train,
    y_train,
    batch_size=BATCH_SIZE)


In [120]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(ROWS, COLS, CHANNELS)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
#     model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2,2)))
#     model.add(Conv2D(256, (3, 3), activation='relu'))
#     model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    # model.add(Dense(1024, activation='relu'))
    # model.add(Dense(1000, activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    return model


In [121]:
model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0